In [39]:
import csv
import itertools
import re
import pandas as pd
from pysolar.solar import *
from pysolar.util import *
# import pysolar
from datetime import timezone
from datetime import datetime


#* ENVS
wspr_csv = "2022-02-05-flt11.csv"
time_slot = 4
re_time_slot = f'^..:.{time_slot}'
parsed_csv = f'parsed_{wspr_csv}'

header = []
parsed_dt = []
alt = []


def alt_calc(power,split_ts):
    if split_ts == None:
        alt_m = int(power)*20 #* Type 3 WSPR message decode x 20 (20m per dBm)
    else:
        alt_m = int(power)*300 #* Type 1 or 2 WSPR message decode x 300 (300m per dBm)
    alt_ft = alt_m * 3.28 #* Convert meters to feet
    return (alt_m, int(alt_ft))


with open(wspr_csv, 'r') as file:
    data = csv.reader(file)
    header = next(data)
    for row in data:
        alt = []
        dt_obj = datetime.strptime(row[1], '%Y-%m-%d %H:%M:%S').replace(tzinfo=timezone.utc)
        solar_altitude = get_altitude(float(row[8]), float(row[9]), dt_obj)
        row.append(solar_altitude)
        # print(solar_altitude)
        split_dt = row[1].split(' ')
        split_ts = re.search(re_time_slot, split_dt[1])
        alt_m = alt_calc(row[15], split_ts)
        alt.extend(alt_m)
        parsed_dt.append(list(itertools.chain(row, split_dt, alt)))


add_headers = ['solar_altitude', 'date', 'time', 'altitude_m', 'altitude_ft']
header.extend(add_headers)

with open(parsed_csv, "w",newline="") as f:
    writer = csv.writer(f)
    header[1] = 'datetime'
    # print(header)
    writer.writerow(header)
    for row in parsed_dt:
        writer.writerow(row)

In [47]:
#* Adding in a dataframe to work to add the WSPR 3 packet to the Type 1 and 2

parsed_df = pd.read_csv(parsed_csv, parse_dates=["datetime"], usecols=['datetime','date','time', 'tx_lat', 'tx_lon', 'tx_loc', 'altitude_m', 'altitude_ft', 'solar_altitude'])
parsed_df.tail()

,datetime,tx_lat,tx_lon,tx_loc,solar_altitude,date,time,altitude_m,altitude_ft
4086,2022-02-05 22:44:00,60.104,-141.792,BP90cc,13.196858,2022-02-05,22:44:00,8100,26568
4087,2022-02-05 22:44:00,60.354,-141.042,BP90li,12.856204,2022-02-05,22:44:00,8100,26568
4088,2022-02-05 22:46:00,60.354,-141.042,BP90li,12.786774,2022-02-05,22:46:00,340,1115
4089,2022-02-05 22:46:00,60.354,-141.042,BP90li,12.786774,2022-02-05,22:46:00,340,1115
4090,2022-02-05 22:46:00,60.354,-141.042,BP90li,12.786774,2022-02-05,22:46:00,340,1115


In [48]:
parsed_df.dtypes

datetime          datetime64[ns]
tx_lat                   float64
tx_lon                   float64
tx_loc                    object
solar_altitude           float64
date                      object
time                      object
altitude_m                 int64
altitude_ft                int64
dtype: object

In [49]:
parsed_df['type'] = parsed_df['time'].str[4]

In [50]:
parsed_df.drop_duplicates(subset ="datetime", keep = False, inplace = True)
parsed_df.shape

(125, 10)

In [51]:
parsed_df

,datetime,tx_lat,tx_lon,tx_loc,solar_altitude,date,time,altitude_m,altitude_ft,type
499,2022-01-27 13:34:00,39.479,-73.042,FM39,14.148198,2022-01-27,13:34:00,8100,26568,4
500,2022-01-27 13:36:00,39.271,-73.958,FM39ag,14.028099,2022-01-27,13:36:00,540,1771,6
685,2022-01-27 14:54:00,39.271,-73.208,FM39jg,24.722647,2022-01-27,14:54:00,8100,26568,4
776,2022-01-27 15:34:00,39.271,-72.792,FM39og,28.696170,2022-01-27,15:34:00,8100,26568,4
777,2022-01-27 15:36:00,39.313,-72.625,FM39qh,28.868466,2022-01-27,15:36:00,600,1967,6
...,...,...,...,...,...,...,...,...,...,...
3957,2022-02-05 02:06:00,34.021,-161.125,AM94ka,23.022112,2022-02-05,02:06:00,340,1115,6
4002,2022-02-05 03:06:00,34.354,-159.458,BM04gi,11.506342,2022-02-05,03:06:00,260,852,6
4009,2022-02-05 18:36:00,56.104,-147.958,BO66ac,5.976207,2022-02-05,18:36:00,260,852,6
4044,2022-02-05 20:46:00,58.521,-145.125,BO78km,14.486568,2022-02-05,20:46:00,200,656,6


In [52]:
parsed_df.to_csv('new_wspr.csv', index = False)

In [60]:
# I only print for type 0 packets and add the 0 detail to type 8 base.
with open("new_wspr.csv", "r") as a_file:
    for line in a_file:
        spline = line.split(',')
        # print(spline[8].strip('\n'))
        if spline[9].strip('\n') == '4':
            # print("it's an 8")
            base_altitude = int(spline[8])
        elif spline[9].strip('\n') == '6':
            altitude = base_altitude + int(spline[8])
            solar_altitude = float(spline[4])
            print(f"{spline[5]} {spline[6]} {altitude} ft, solar altitude {round(solar_altitude)} degrees")

2022-01-27 13:36:00 28339 ft, solar altitude 14 degrees
2022-01-27 15:36:00 28535 ft, solar altitude 29 degrees
2022-01-27 15:56:00 28535 ft, solar altitude 30 degrees
2022-01-27 16:16:00 28339 ft, solar altitude 31 degrees
2022-01-28 14:16:00 19352 ft, solar altitude 20 degrees
2022-01-28 14:36:00 17843 ft, solar altitude 21 degrees
2022-01-28 14:56:00 18040 ft, solar altitude 22 degrees
2022-01-29 11:56:00 27683 ft, solar altitude 10 degrees
2022-01-29 12:16:00 27683 ft, solar altitude 11 degrees
2022-01-29 13:06:00 27683 ft, solar altitude 12 degrees
2022-01-29 13:26:00 27683 ft, solar altitude 12 degrees
2022-01-30 07:16:00 26764 ft, solar altitude 8 degrees
2022-01-30 08:16:00 26764 ft, solar altitude 12 degrees
2022-01-30 09:16:00 26764 ft, solar altitude 14 degrees
2022-01-30 09:36:00 26764 ft, solar altitude 15 degrees
2022-01-30 10:16:00 26764 ft, solar altitude 14 degrees
2022-01-31 08:06:00 27027 ft, solar altitude 16 degrees
2022-01-31 08:26:00 27027 ft, solar altitude 18 d

Now want to take the altitude in the :08 mins and add to the following :00 mins. to get teh precise altitude.

```
date,time,altitude_m,altitude_ft
2022-01-07,23:38:00,6900,22632
2022-01-07,23:40:00,1060,3476
```

For example the precise altitude is `7960 m`